# Managed Spot Training for XGBoost

---

## Normal Conf

In [9]:
import sys
!{sys.executable} -m pip install -qU awscli boto3 "sagemaker>=1.71.0,<2.0.0"

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.4.2 requires botocore<1.27.60,>=1.27.59, but you have botocore 1.29.140 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [10]:
%%time

import io
import os
import boto3
import sagemaker
import urllib

role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sess = sagemaker.Session()

bucket = "test-sagemaker-examples-1357942113492"
prefix = "DEMO-Spot"

CPU times: user 272 ms, sys: 51.8 ms, total: 324 ms
Wall time: 637 ms


In [11]:
%%time

FILE_DATA = "abalone"
urllib.request.urlretrieve("https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/regression/abalone", FILE_DATA)
sess.upload_data(FILE_DATA, bucket=bucket, key_prefix=prefix + '/train')

CPU times: user 66 ms, sys: 14.1 ms, total: 80.1 ms
Wall time: 1.83 s


's3://test-sagemaker-examples-1357942113492/DEMO-Spot/train/abalone'

In [12]:
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(region, "xgboost", "1.0-1")

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


FileNotFoundError: [Errno 2] No such file or directory: '/opt/conda/lib/python3.7/site-packages/sagemaker/image_uri_config/xgboost.json'

In [7]:
hyperparam = {
    "max_depth":"5",
    "eta":"0.2",
    "gamma":"4",
    "min_child_weight":"6",
    "subsample":"0.7",
    "silent":"0",
    "objective":"reg:squarederror",
    "num_round":"50"
}

instance_type = "ml.m5.2xlarge"
output_path = f's3://{bucket}/{prefix}/abalone-xgb/output'
content_type = "libsvm"

In [8]:
import time
from time import gmtime, strftime

job_name = f"jumpstart-example-xgboost-spot-{strftime('%Y-%m-%d-%H-%M-%S', gmtime())}"

print("Training job", job_name)

train_use_spot = True
train_max_run = 3600
train_max_wait = 7200 if train_use_spot else None
checkpoint_s3_uri = (f's3://{bucket}/{prefix}/checkpoints/{job_name}' if train_use_spot else none)

print("Checkpoint path:", checkpoint_s3_uri)

estimator = sagemaker.estimator.Estimator(
    container,
    role,
    hyperparameters=hyperparam,
    train_instance_count=1,
    train_instance_type=instance_type,
    train_volume_size=5,
    output_path=output_path,
    sagemaker_session=sess,
    train_use_spot_instances=train_use_spot,
    train_max_run=train_max_run,
    train_max_wait=train_max_wait,
    checkpoint_s3_uri=checkpoint_s3_uri
)

train_input = sagemaker.s3_input(
    s3_data=f's3://{bucket}/{prefix}/train', 
    content_type='libsvm')
estimator.fit({'train': train_input}, job_name=job_name)

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_run has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_use_spot_instances has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_wait has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_volume_size has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Training job jumpstart-example-xgboost-spot-2023-05-25-08-59-54
Checkpoint path: s3://test-sagemaker-examples-1357942113492/DEMO-Spot/checkpoints/jumpstart-example-xgboost-spot-2023-05-25-08-59-54


AttributeError: module 'sagemaker' has no attribute 's3_input'